In [1]:
import torchxrayvision as xrv

rsna = xrv.datasets.RSNA_Pneumonia_Dataset('/home/zuzanna/rsna/test',views=["PA","AP"])  

/home/zuzanna/miniconda3/envs/medklip/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/zuzanna/miniconda3/envs/medklip/lib/python3.9/site-packages/torchxrayvision/utils.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import ruamel.yaml as yaml

yaml = yaml.YAML(typ='rt')
config = yaml.load(open('/home/zuzanna/MedKLIP/Sample_Finetuning_SIIMACR/I1_classification/configs/baseline.yaml', 'r'))

In [3]:
from models.tokenization_bert import BertTokenizer
from sklearn.metrics import roc_auc_score,precision_recall_curve,accuracy_score

def compute_AUCs(gt, pred, n_class):
    AUROCs = []
    gt_np = gt.cpu().numpy()
    pred_np = pred.cpu().numpy()
    
    # Handle single class scenario
    if n_class == 1:
        AUROCs.append(roc_auc_score(gt_np, pred_np))
    else:
        for i in range(n_class):
            AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
    
    return AUROCs

def get_tokenizer(tokenizer,target_text):
    
    target_tokenizer = tokenizer(list(target_text), padding='max_length', truncation=True, max_length= 64, return_tensors="pt")
    
    return target_tokenizer

In [6]:
import json
import torch.nn as nn
import torch
torch.cuda.init()
from models.model_MedKLIP import MedKLIP

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

json_book = json.load(open('/home/zuzanna/MedKLIP/Sample_Finetuning_SIIMACR/I1_classification/data_file/observation explanation.json','r'))
disease_book = [json_book[i] for i in json_book]
ana_book = [ 'It is located at ' + i for i in ['trachea', 'left_hilar', 'right_hilar', 'hilar_unspec', 'left_pleural',
        'right_pleural', 'pleural_unspec', 'heart_size', 'heart_border', 'left_diaphragm',
        'right_diaphragm', 'diaphragm_unspec', 'retrocardiac', 'lower_left_lobe', 'upper_left_lobe',
        'lower_right_lobe', 'middle_right_lobe', 'upper_right_lobe', 'left_lower_lung', 'left_mid_lung', 'left_upper_lung',
        'left_apical_lung', 'left_lung_unspec', 'right_lower_lung', 'right_mid_lung', 'right_upper_lung', 'right_apical_lung',
        'right_lung_unspec', 'lung_apices', 'lung_bases', 'left_costophrenic', 'right_costophrenic', 'costophrenic_unspec',
        'cardiophrenic_sulcus', 'mediastinal', 'spine', 'clavicle', 'rib', 'stomach', 'right_atrium', 'right_ventricle', 'aorta', 'svc',
        'interstitium', 'parenchymal', 'cavoatrial_junction', 'cardiopulmonary', 'pulmonary', 'lung_volumes', 'unspecified', 'other']]
tokenizer = BertTokenizer.from_pretrained(config['text_encoder'])
ana_book_tokenizer = get_tokenizer(tokenizer,ana_book).to(device)
disease_book_tokenizer = get_tokenizer(tokenizer,disease_book).to(device)

print("Creating model")
model = MedKLIP(config,ana_book_tokenizer, disease_book_tokenizer)
model = nn.DataParallel(model, device_ids = [i for i in range(torch.cuda.device_count())])
model = model.to(device)  

Creating model


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


text feature extractor: emilyalsentzer/Bio_ClinicalBERT


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 11.90 GiB total capacity; 428.13 MiB already allocated; 33.62 MiB free; 484.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
import torch

checkpoint = torch.load('/home/zuzanna/MedKLIP/output/deformable_pretrain/checkpoint_state.pth', map_location='cpu')
# checkpoint = torch.load('/home/zuzanna/MedKLIP/output/deformable_no_pretrain/best_valid.pth', map_location='cpu')

/home/zuzanna/miniconda3/envs/medklip/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: can't set attribute